## Gradio

[Gradio](https://www.gradio.app) can enable simple web interfaces to your software. In this example, we are using Gradio to get a simple chat interface to a large language model.

In [1]:
%%writefile gradio_example.py
# Import necessary libraries
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
import gradio as gr
import os
import random
import socket
import sys

ip = socket.gethostbyname(socket.gethostname())
hostname = socket.gethostname().split('.')[0]
port = random.randint(10000, 50000)
trainee_user = os.environ['TRAINEE_USERNAME']

print('Open the following URL in your webbrowser:')
print(f'https://hpctraining.org/{trainee_user}/proxy/absolute/{hostname}:{port}/')
print('')
sys.stdout.flush()

# model_name = "meta-llama/Llama-3.2-1B-Instruct"
model_name = "/leonardo_scratch/fast/EUHPC_D20_063/huggingface/models/meta-llama--Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='cuda',
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
)
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer)
def get_answer(question, history=[]):
    history.append(
        {'role': 'user', 'content': question}
    )
    result = pipe(history, max_new_tokens=500, return_full_text=False)
    return result[0]['generated_text'].strip()
    # return question

chat_interface = gr.ChatInterface(get_answer, type='messages')
chat_interface.launch(share=False, server_name=ip, server_port=port, root_path=f'/{trainee_user}/proxy/absolute/{hostname}:{port}')

Overwriting gradio_example.py


In [2]:
%%writefile run_gradio_example.slurm
#!/bin/bash

#SBATCH --partition=boost_usr_prod
# #SBATCH --qos=boost_qos_dbg
#SBATCH --account=EUHPC_D20_063
#SBATCH --reservation=s_tra_ncc

## Specify resources:
## Leonardo Booster: 32 CPU cores and 4 GPUs per node => request 8 * number of GPUs CPU cores
## Leonardo Booster: 512 GB in total => request approx. 120 GB * number of GPUs requested
#SBATCH --nodes=1
#SBATCH --gpus-per-task=1  # up to 4 on Leonardo
#SBATCH --ntasks-per-node=1  # always 1
#SBATCH --mem=120GB  # should be 120GB * gpus-per-task on Leonardo
#SBATCH --cpus-per-task=8  # should be 8 * gpus-per-task on Leonardo

#SBATCH --time=0:10:00

# Include commands in output:
set -x

# Print current time and date:
date

# Print host name:
hostname

# List available GPUs:
nvidia-smi

# Construct command to run container:
export CONTAINER="singularity run --nv --home=$HOME $SINGULARITY_CONTAINER"

# Run AI scripts:
$CONTAINER python3 gradio_example.py

Overwriting run_gradio_example.slurm


Now submit the SLURM job:

In [3]:
!sbatch --job-name=$TRAINEE_USERNAME run_gradio_example.slurm

Submitted batch job 19769276


Execute `squeue` to see, if your job is already running:

In [4]:
!squeue --name=$TRAINEE_USERNAME 

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          19769276 boost_usr   martin mpfister  R       0:04      1 lrdn1321


Once your job is running, look at the output of the job using the following command (replace the number with the JOBID from above):

In [7]:
!cat slurm-19769276.out

+ date
Tue Sep  9 15:06:17 CEST 2025
+ hostname
lrdn1321.leonardo.local
+ nvidia-smi
Tue Sep  9 15:06:17 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM-64GB           On  | 00000000:56:00.0 Off |                    0 |
| N/A   43C    P0              60W / 458W |      2MiB / 65536MiB |      0%      Default |
|                                         |                      |       

Finally, when you are finished, please cancel the SLURM job to free the resources:

In [8]:
!scancel 19769276

If you want to, you can also delete the files that we create above:

In [9]:
!rm gradio_example.py run_gradio_example.slurm slurm-*.out